TempDatabaseService
---

The TempDatabaseService is used to create an ephemeral database. It's \_\_init\_\_ expects a `url` to connect to an existing database instance. In some cases the TempDatabaseService can be configured to create a database instance instead of connecting to an existing instance (e.g. using `psql` if `postgresql` is installed).

In this example we will use **docker** to create a **postgres** instance to connect to. This pattern can be leveraged in CI pipelines to provide a database instance for testing.

_Note: TempDatabaseService is not a construct of the knockoff.sdk and is a utility that can be used to create an ephemeral database that the knockoff.sdk connects to and populates with data configured for the various test cases._


### Create database instance

The following docker command creates a postgres instance (url=`postgresql://postgres@localhost:5432/postgres`)

`docker run --rm  --name pg-docker -e POSTGRES_HOST_AUTH_METHOD=trust -d -p 5432:5432  postgres:11.9`


### Instantiate TempDatabaseService

In [1]:
from knockoff.tempdb.db import TempDatabaseService
from knockoff.tempdb.setup_teardown import postgres_setup_teardown

url = "postgresql://postgres@localhost:5432/postgres"
temp_db_service = TempDatabaseService(
    url,
    setup_teardown=postgres_setup_teardown
)

### setup_teardown

The setup_teardown parameter expects a generator function that takes a url and returns a generator that yields a single value, the url of the temp database created in the database instance. The default setup_teardown is the **postgres_setup_teardown**.

Below is an example pseudo implementation of a setup_teardown generator function.

```
def some_setup_teardown(url):
    
    # do some setup (optional)
    # connect to the database instance at url
    # create a new database at temp_url
    temp_url = ... # url for temp database (we recommend using a uid for a unique name to avoid collisions)
    
    # provides the temp_url to the TempDatabaseService
    yield temp_url
    
    # do some teardown (optional)
```

### start the service

`start()` will create the temp database and return the url to the caller. The caller can use the `temp_url` to configure the application or services to connect to the temp database.

In [2]:
temp_url = temp_db_service.start()
print(temp_url)

postgresql://postgres@localhost:5432/test_1e82c2a4a322428cbbefb980f40c2f00


### stop the service

`stop()` will destroy the temp database.

In [3]:
temp_db_service.stop()

### initialize_tables

The application can use the temp_url to make a connection and initialize it's tables. However, this can also be performed by the TempDatabaseService by providing a callable that takes a url to the `initialize_tables` parameter in the \_\_init\_\_. This callable should connect to the database with the temp_url and initialize the applications tables. By default the initialize_tables parameter is None and TempDatabaseService will not attempt to make the `initialize_tables(temp_url)` call. 


### SqlAlchemyInitTablesFunc
(`knockoff.tempdb.initialize_tables:SqlAlchemyInitTablesFunc`)

SqlAlchemyInitTablesFunc accepts a sqlalchemy declarative base in its \_\_init\_\_ which creates a callable that takes a url and initializes the tables using the metadata from the base.

### pytest fixture

This class can be used as a pytest fixture to provide ephemeral databases for each unit test using the following pattern.

In [4]:
import pytest
from sqlalchemy import create_engine

from knockoff.tempdb.db import TempDatabaseService
from knockoff.tempdb.setup_teardown import postgres_setup_teardown
from knockoff.tempdb.initialize_tables import SqlAlchemyInitTablesFunc

@pytest.fixture(scope="function")
def empty_db_engine():
    """
    This fixture returns a sqlalchemy 
    engine to connect to an 
    an empty temp database
    """

    # setup

    url = .. # application provides some way to create the url
    base = .. # application provides the sqlalchemy declarative base with
              # metadata to initialize tables with it's data model

    temp_db_service = TempDatabaseService(
        url,
        setup_teardown=postgres_setup_teardown,
        initialize_tables=SqlAlchemyInitTablesFunc(base)
    )

    # setup the database and initialize the tables
    temp_url = temp_db_service.start()

    # application should configure itself to connect to the temp_url

    engine = create_engine(new_url)
    yield engine

    # tear down
    temp_db_service.stop()
